In [24]:
import os
import pandas as pd
from pathlib import Path
import re
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.optimizers import Adam
import warnings 
warnings.filterwarnings(action='ignore')

ImportError: DLL load failed while importing lapack_lite: 지정된 모듈을 찾을 수 없습니다.

In [14]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [25]:
pip install --upgrade tensorflow scikit-learn


  Using cached tensorflow-2.13.0-cp39-cp39-win_amd64.whl (1.9 kB)
  Using cached tensorflow_intel-2.13.0-cp39-cp39-win_amd64.whl (276.5 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.12.0
    Uninstalling tensorflow-estimator-2.12.0:
      Successfully uninstalled tensorflow-estimator-2.12.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.12.3
    Uninstalling tensorboard-2.12.3:
      Successfully uninstalled tensorboard-2.12.3
  Attempting uninstall: keras
    Found existing installation: keras 2.12.0
    Uninstalling keras-2.12.0:
      Successfully uninstalled keras-2.12.0
  Attempting uninstall: tensorflow-intel
    Found existing installation: tensorflow-intel 2.12.0
    Uninstalling tensorflow-intel-2.12

ERROR: Could not install packages due to an OSError: [WinError 5] 액세스가 거부되었습니다: 'C:\\Users\\User\\miniconda3\\Lib\\site-packages\\~ensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [3]:
#파일 불러오기
train = pd.read_csv('train.csv', encoding = 'utf-8')
test = pd.read_csv('test_x.csv', encoding = 'utf-8')
#sample_submission = pd.read_csv('sample_submission.csv', encoding = 'utf-8')

In [4]:
train.shape, test.shape


((54879, 3), (19617, 2))

In [6]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # Restrict TensorFlow to only use the first GPU
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
    except RuntimeError as e:
        # Visible devices must be set before GPUs have been initialized
        print(e)
else:
    print('No GPU detected')

No GPU detected


# 모델 생성

In [ ]:
#RNN생성

import torch.nn as nn#함수 사용

# cpu/gpu설정
device="cuda"if torch.cuda.is_available() else "cpu"
print(f"using {device} device")


class Net(nn.Module):
    def __init__(self,input_size,hidden_size,output_size):
        super(Net,self).__init__()
        self.hidden_size=hidden_size
        self.rnn=nn.RNN(input_size=56,hidden_size=hidden_size)
        self.fc=nn.Linear(hidden_size,output_size)
    
    def forward(self,input,hidden):#input, hidden 둘다 필요!
        print(input.size())
        x=input.permute(1,0,2).contiguous()#메모리도 sequence하게 가져와라
        #시계열에서는 필수
        out,h=self.rnn(x,hidden)
        print(out.size())
        x=self.fc(out[-1])
        return x,h
        
    def initHidden(self):# 초기 히든값 1로 구성
        return torch.ones(1,1,self.hidden_size)

n_hidden=128
rnn=Net(n_letters,n_hidden,n_categories).to(device)
  